In [ ]:
from flask import Flask
from werkzeug.serving import run_simple

# Initializes the Flask app
app = Flask(__name__) # This creates an instance of the Flask class and assigns it to a variable. It's like creating a blueprint for the app

# Define's a route
@app.route("/") # This is a route director. It tells Flask which URL to connect to this function
def home(): # This function runs when someone visits the specified route
    return "Hello, Flask!"

# Run the app
run_simple("localhost", 5000, app)